### Load libraries & config

In [1]:
from datetime import date, datetime
import girder_client as gc
import json
import os
from urllib.parse import quote
import random
import re
import string
import tzlocal
from urllib.request import urlopen

with open("config.json", "r") as fp:
    config=json.load(fp)

### Connect to Girder

In [2]:
girder_connection = gc.GirderClient(
    apiUrl="{}/api/v1/".format(
        config["girder-production"]["host"]
    )
)
girder_connection.authenticate(
    username=config["girder-production"]["user"],
    password=config["girder-production"]["password"]
)

In [3]:
activity_sets = girder_connection.get(
    "folder?parentType=collection&parentId={}".format(
        girder_connection.get(
            "collection?name=Volumes"
        )[0]["_id"]
    )
)

### Authorize Viewers to View their Users as Authorized in Admin Panel

In [4]:
for a in activity_sets:
    print("Activity Set: {}".format(a['name']))
    for v in a['meta']['members']['viewers']:
        print("Viewer: {}".format(girder_connection.getUser(v)['firstName']))
        for u in a['meta']['members']['viewers'][v]:
            try: 
                access = girder_connection.get("user/{}/access".format(u))
            except:
                access = {'users': []}
            print("User: {}".format(girder_connection.getUser(u)['firstName']))
            print("access: {}".format(", ".join([user['name'] for user in access['users']])))
            if (v not in [user['id'] for user in access['users']]):
                access['users'].append({'id': v, 'level': 0})
                girder_connection.put("user/{}/access".format(u),
                    data={"access":json.dumps(access)}
                )
            try:
                responses = girder_connection.get(
                    "folder?parentType=user&parentId={}&name=Responses".format(
                        u
                    )
                )
                for responses_folder in responses:
                    print("Responses: {}".format(responses_folder['name'] if 'name' in responses_folder else responses_folder))
                    try:
                        access = girder_connection.get(
                            "folder/{}/access".format(
                                responses_folder['_id']
                            )
                        )
                        if v not in [uid["id"] for uid in access['users']]:
                            access['users'].append({'id': v, 'level': 0})
                        girder_connection.put(
                            'folder/{}/access'.format(
                                responses_folder['_id']
                            ),
                            data={
                                "access":json.dumps(access),
                                "recurse":False
                            },
                        )
                        print(access)
                        activity_set_responses = girder_connection.get(
                            "folder?parentType=folder&parentId={}&name={}".format(
                                responses_folder['_id'],
                                a['name']
                            )
                        )
                        for activity_set_response_folder in activity_set_responses: 
                            print("Responses Folder: {}".format(activity_set_response_folder['name']))
                            access = girder_connection.get(
                                "folder/{}/access".format(
                                    activity_set_response_folder['_id']
                                )
                            )
                            print("access: {}".format(", ".join([uid["name"] for uid in access['users']])))
                            if v not in [uid["id"] for uid in access['users']]:
                                access['users'].append({'id': v, 'level': 0})
                            girder_connection.get('folder/{}'.format(activity_set_response_folder['_id']))['name']
                            girder_connection.put(
                                'folder/{}/access'.format(
                                    activity_set_response_folder['_id']
                                ),
                                data={
                                    "access":json.dumps(access),
                                    "recurse":True
                                },
                            )
                    except:
                        activity_set_responses = None
            except:
                responses = None
        print("\n")

Activity Set: A activity set of various sleep assessments
Viewer: anisha
User: anisha
access: anisha keshavan, Miriam San Lucas, Jon Clucas, Jake Son, Jake Son, Erik P, Erik Ilyin3, Camille Gregory, Germeen Kilada, Lindsay Alexander, Victoria Castagna, Jasmine Escalera, Minji Kang, Arno Klein, Batya Septimus, Giuseppe Minniti, Kristen Schiavo, Grace McAlindin, Madeline Walsh, Jacob Derosa, Jacob DeRosa, Chase Hill
Responses: Responses
{'groups': [], 'users': [{'email': 'anishakeshavan@gmail.com', 'flags': [], 'id': '5bdb6a97336da80de9145e35', 'level': 2, 'login': 'akeshavan', 'name': 'anisha keshavan'}, {'email': 'jon.clucas@childmind.org', 'flags': [], 'id': '5b4ca32338488311497dc9bc', 'level': 0, 'login': 'clucasjon', 'name': 'Jon Clucas'}, {'email': 'arno@childmind.org', 'flags': [], 'id': '5b634cbf0fc13a0df633ae00', 'level': 0, 'login': 'arnoatchildmind.org', 'name': 'Arno Klein'}]}


Activity Set: activity sets relating to exercise
Activity Set: Anisha test #4
Activity Set: Anisha

### Authorize Managers to View Users as Authorized in Admin Panel

In [5]:
for a in activity_sets:
    for u in a['meta']['members']['users']:
        try:
            access = girder_connection.get(
                'user/{}/access'.format(u)
            )
            for m in a['meta']['members']['managers']:
                if m not in [uid['id'] for uid in access['users']]:
                    print(m)
                    access['users'].append({'id': m, 'level': 0})
            print(access)
            girder_connection.put(
                'user/{}/access'.format(u),
                data={"access":json.dumps(access)}
            )
        except:
            access = {}

{'groups': [], 'users': [{'email': 'anishakeshavan@gmail.com', 'flags': [], 'id': '5bdb6a97336da80de9145e35', 'level': 2, 'login': 'akeshavan', 'name': 'anisha keshavan'}, {'email': 'miriam.sanlucas@childmind.org', 'flags': [], 'id': '5c2fcfebc25b5a625531c8fd', 'level': 0, 'login': 'miriam.sanlucas', 'name': 'Miriam San Lucas'}, {'email': 'jon.clucas@childmind.org', 'flags': [], 'id': '5b4ca32338488311497dc9bc', 'level': 0, 'login': 'clucasjon', 'name': 'Jon Clucas'}, {'email': 'j.son5457@gmail.com', 'flags': [], 'id': '5bd87d09336da80de9145b23', 'level': 0, 'login': 'js545', 'name': 'Jake Son'}, {'email': 'jake.son@childmind.org', 'flags': [], 'id': '5b7f1cf7e534fe32c5f5db8a', 'level': 0, 'login': 'jake-son', 'name': 'Jake Son'}, {'email': 'erik2@test.com', 'flags': [], 'id': '5b634cbd0fc13a0df633adfd', 'level': 0, 'login': 'erik2attest.com', 'name': 'Erik P'}, {'email': 'erik.ilyin@aol.com', 'flags': [], 'id': '5b4e1f1f8f5b186be929676d', 'level': 0, 'login': 'erik', 'name': 'Erik Ily

### Authorize Healthy Brain Network Data Access Group Data Access to All **EMA: Parent** Activity Data

In [6]:
for user in girder_connection.getFolder('5bd87caa336da80de9145af2')['meta']['members']['users']:
    try:
        responses = girder_connection.get(
            "folder?parentType=user&parentId={}&name=Responses".format(
                user
            )
        )
        for responses_folder in responses:
            try:
                ema_parent_responses = girder_connection.get(
                    "folder?parentType=folder&parentId={}&name=EMA%3A%20Parent".format(
                        responses_folder['_id']
                    )
                )
                for ema_parent_response_folder in ema_parent_responses: 
                    access = girder_connection.get(
                        "folder/{}/access".format(
                            ema_parent_response_folder['_id']
                        )
                    )
                    if "5bab8fc80a76ac049becd5ec" not in [gid["id"] for gid in access['groups']]:
                        access['groups'].append({'id': '5bab8fc80a76ac049becd5ec', 'level': 0})
                print(access)
                girder_connection.put(
                    'folder/{}/access'.format(
                        ema_parent_response_folder['_id']
                    ),
                    data={
                        "access":json.dumps(access),
                        "recurse":True
                    },
                )
            except:
                ema_parent_responses = None
    except:
        responses = None

{'groups': [{'description': 'Individuals associated with the Healthy Brain Network who have blanket access to participant data.', 'flags': [], 'id': '5bab8fc80a76ac049becd5ec', 'level': 0, 'name': 'Healthy Brain Network data access'}], 'users': [{'email': 'jon.clucas@childmind.org', 'flags': [], 'id': '5b4ca32338488311497dc9bc', 'level': 2, 'login': 'clucasjon', 'name': 'Jon Clucas'}, {'email': 'anishakeshavan@gmail.com', 'flags': [], 'id': '5bdb6a97336da80de9145e35', 'level': 0, 'login': 'akeshavan', 'name': 'anisha keshavan'}, {'email': 'paul.baier@gmail.com', 'flags': [], 'id': '5c1c29e4c25b5a6a94d5fc13', 'level': 0, 'login': 'paulbaier', 'name': 'Paul Baier'}, {'email': 'anirudh.krishnakumar@childmind.org', 'flags': [], 'id': '5b634cc20fc13a0df633ae05', 'level': 0, 'login': 'anirudh.krishnakumaratchildmind.org', 'name': 'Anirudh Krishnakumar'}]}
{'groups': [{'description': 'Individuals associated with the Healthy Brain Network who have blanket access to participant data.', 'flags':

### Consolidate duplicated permissions

In [7]:
for u in girder_connection.get('user'):
    access = girder_connection.get(
        'user/{}/access'.format(u['_id'])
    )
    new_access = {'groups': access['groups'], 'users': []}
    for au in access['users']:
        if au['id'] not in [uid['id'] for uid in new_access['users']]:
            new_access['users'].append(au)
    print(new_access)
    girder_connection.put(
        'user/{}/access'.format(u["_id"]),
        data={"access":json.dumps(new_access)}
    )

{'groups': [], 'users': [{'email': 'sanuann@mit.edu', 'flags': [], 'id': '5c4789c145e8fc71d167bd0a', 'level': 2, 'login': 'sanuann', 'name': 'Sanu Ann Abraham'}]}
{'groups': [], 'users': [{'email': 'louonline1969@gmail.com', 'flags': [], 'id': '5c65d3b37c7e6d0e0e770685', 'level': 2, 'login': 'ethan.addonizio', 'name': 'Ethan Addonizio'}]}
{'groups': [], 'users': [{'email': 'admin@mindlogger.org', 'flags': [], 'id': '5b634cbe0fc13a0df633adfe', 'level': 2, 'login': 'adminatmindlogger.org', 'name': 'Super Admin'}, {'email': 'jon.clucas@childmind.org', 'flags': [], 'id': '5b4ca32338488311497dc9bc', 'level': 0, 'login': 'clucasjon', 'name': 'Jon Clucas'}]}
{'groups': [], 'users': [{'email': 'admin@ab2cd.com', 'flags': [], 'id': '5b634cc00fc13a0df633ae02', 'level': 2, 'login': 'adminatab2cd.com', 'name': 'Admin Admin'}]}
{'groups': [], 'users': [{'email': 'admin@mindlogger.info', 'flags': [], 'id': '5c1411e0c25b5a144b269e63', 'level': 2, 'login': 'adminatmindlogger', 'name': 'Admin Admin'}]}

### Purge non-existent users from admin panel

In [8]:
all_assignees = list({
    *{
        e for a in
        activity_sets for e in
        a['meta']['members']['editors']
    },
    *{
        m for a in
        activity_sets for m in
        a['meta']['members']['managers']
    },
    *{
        u for a in
        activity_sets for u in
        a['meta']['members']['users']
    },
    *{
        v for a in
        activity_sets for v in
        a['meta']['members']['viewers']
    },
    *{
        u for a in
        activity_sets for v in
        a['meta']['members']['viewers'] for u in
        a['meta']['members']['viewers'][v] 
    }
})
purge_list = []
for assignee in all_assignees:
    try:
        girder_connection.getUser(assignee)
    except:
        purge_list.append(assignee)
print(purge_list)
for a in activity_sets:
    original_members = a['meta']['members'].copy()
    for role in ['editors', 'users', 'managers']:
        a['meta']['members'][role] = list(set(a['meta']['members'][role]) - set(purge_list))
    for purgee in purge_list:
        a['meta']['members']['viewers'].pop(purgee, None)
    for v in a['meta']['members']['viewers']:
        a['meta']['members']['viewers'][v] = list(set(a['meta']['members']['viewers'][v]) - set(purge_list))
    if(original_members != a['meta']['members']):
        girder_connection.put(
            'folder/{}'.format(
                a['_id']
            ),
            data={
                'metadata':json.dumps(
                    a['meta']
                )
            }
        )

[]


In [9]:
for manager in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['managers']:
    u = girder_connection.getUser(manager)
    print("{} {}".format(u['firstName'], u['lastName']))

IndexError: list index out of range

In [ ]:
all_assignees = list({
    *{
        e for a in
        activity_sets for e in
        a['meta']['members']['editors']
    },
    *{
        m for a in
        activity_sets for m in
        a['meta']['members']['managers']
    },
    *{
        u for a in
        activity_sets for u in
        a['meta']['members']['users']
    },
    *{
        v for a in
        activity_sets for v in
        a['meta']['members']['viewers']
    },
    *{
        u for a in
        activity_sets for v in
        a['meta']['members']['viewers'] for u in
        a['meta']['members']['viewers'][v] 
    }
})
for assignee in all_assignees:
    if assignee not in [a for a in activity_sets if a['name'] == "EMA: Parent"][0]['meta']['members']['users']:
        u = girder_connection.getUser(assignee)
        print("{} {} {}".format(u['firstName'], u['lastName'], u['_id']))